# Task
Implement an AI-Based Disaster Detection and Rescue Support system using the C2A dataset. The system detect multiple natural disasters (floods, earthquakes, tsunamis, fires) and the number of people stuck, and based on this information, provide support team recommendations to enhance disaster response efficiency and support affected communities. The system quickly identifies disaster events and suggests appropriate emergency resources like rescue teams, medical aid, shelters, food, and evacuation plans. The goal is to enhance disaster response efficiency, reduce harm, and support affected communities in recovery through timely, tailored resource recommendations." The dataset is located at "/content/drive/MyDrive/C2A.zip".

## Setup environment 

### Subtask:
Install necessary libraries and mount Google Drive to access the dataset.


In [1]:
!pip install ultralytics
#from google.colab import drive
#drive.mount('/content/drive')

## Load and extract dataset

### Subtask:
Load the zipped C2A dataset from Google Drive and extract its contents to a local directory.


In [2]:
import zipfile
import os

# Define the corrected path to the zipped dataset in Google Drive
zip_path = 'C:/Users/Admin/Downloads/archive (2).zip'

# Define the path to the local directory where the dataset will be extracted
extract_path = 'D:/path/to/c2a_dataset'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Use the zipfile module to open and extract the contents of the zipped dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset extracted to: {extract_path}")

Dataset extracted to: D:/path/to/c2a_dataset


## Data preparation and preprocessing for human detection

### Subtask:
Define the classes for object detection (initially 'human'), organize the extracted image and annotation files, convert annotations to the YOLO format, split the dataset into training and validation sets, and prepare data loaders for the YOLO model.


In [2]:
import json
import os

def coco_to_yolo(coco_bbox, image_width, image_height):
    """Converts COCO bounding box [x_min, y_min, width, height] to YOLO format [x_center, y_center, width, height]."""
    x_min, y_min, width, height = coco_bbox
    x_center = (x_min + width / 2) / image_width
    y_center = (y_min + height / 2) / image_height
    w_norm = width / image_width
    h_norm = height / image_height
    return [x_center, y_center, w_norm, h_norm]

def convert_annotations(json_path, image_dir, output_label_dir):
    """Converts COCO annotations in a JSON file to YOLO format .txt files."""
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    # Create output directory if it doesn't exist
    os.makedirs(output_label_dir, exist_ok=True)

    # Create a dictionary to map image_id to image information (including width and height)
    image_info = {img['id']: (img['width'], img['height'], img['file_name']) for img in coco_data['images']}

    # Create a dictionary to store annotations per image
    annotations_by_image = {}
    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        if image_id not in annotations_by_image:
            annotations_by_image[image_id] = []
        annotations_by_image[image_id].append(ann)

    # Process annotations for each image
    for image_id, annotations in annotations_by_image.items():
        img_width, img_height, file_name = image_info[image_id]
        # YOLO label file name should match the image file name (but with .txt extension)
        label_file_name = os.path.splitext(file_name)[0] + '.txt'
        label_file_path = os.path.join(output_label_dir, label_file_name)

        with open(label_file_path, 'w') as f:
            for ann in annotations:
                # Assuming 'human' is category_id 0 as observed in the sample annotation
                category_id = ann['category_id'] # This should be 0 for 'human'
                yolo_bbox = coco_to_yolo(ann['bbox'], img_width, img_height)
                # Write the YOLO format: class_index center_x center_y width height
                f.write(f"{category_id} {yolo_bbox[0]:.6f} {yolo_bbox[1]:.6f} {yolo_bbox[2]:.6f} {yolo_bbox[3]:.6f}\n")

    print(f"Converted annotations from {json_path} to YOLO format in {output_label_dir}")

# Define paths
base_dataset_path = 'D:/path/to/c2a_dataset'
train_json_path = os.path.join(r"D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/train/train_annotations.json")
val_json_path = os.path.join(r"D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/val/val_annotations.json")

train_image_dir = os.path.join(r"D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/train/images")
val_image_dir = os.path.join(r"D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/val/images")

train_label_dir = os.path.join(r"D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/train/labels")
val_label_dir = os.path.join(r"D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/val/labels")

# Convert train and validation annotations
convert_annotations(train_json_path, train_image_dir, train_label_dir)
convert_annotations(val_json_path, val_image_dir, val_label_dir)
print("Annotations converted to YOLO format successfully.")
print("Train labels directory:", train_label_dir)
print("Validation labels directory:", val_label_dir)


Converted annotations from D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/train/train_annotations.json to YOLO format in D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/train/labels
Converted annotations from D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/val/val_annotations.json to YOLO format in D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/val/labels
Annotations converted to YOLO format successfully.
Train labels directory: D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/train/labels
Validation labels directory: D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/val/labels


In [3]:
#Create the `data.yaml` configuration file required by YOLO, specifying the paths to the training and validation image and label directories,
#the number of classes, and the class names.


import yaml

# Define the content of the data.yaml file
data_yaml_content = {
    'train': os.path.join(r"D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/train/images"), # Path to training images
    'val': os.path.join(r"D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/val/images"),     # Path to validation images
    'nc': 1,                                                     # Number of classes
    'names': ['human']                                           # Class names
}

# Define the path where the data.yaml file will be saved
data_yaml_path = os.path.join(base_dataset_path, 'data.yaml')

# Write the content to the data.yaml file
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml_content, f, default_flow_style=None)

print(f"Created data.yaml file at: {data_yaml_path}")

# Display the content of the created data.yaml file
with open(data_yaml_path, 'r') as f:
    print("\nContent of data.yaml:")
    print(f.read())

Created data.yaml file at: D:/path/to/c2a_dataset\data.yaml

Content of data.yaml:
names: [human]
nc: 1
train: D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/train/images
val: D:/path/to/c2a_dataset/C2A_Dataset/new_dataset3/val/images



## Model building and Training: human detection

### Subtask:
 Add YOLO model architecture (e.g., YOLOv8) and initialize the YOLO model, potentially using pre-trained weights.


In [5]:
from ultralytics import YOLO

# Choose a YOLOv8 model architecture (e.g., 'yolov8n.pt' for nano model with pre-trained weights)
model_architecture = 'yolov8n.pt'

# Instantiate the YOLO model
model = YOLO(model_architecture)

# Print the model to inspect its layers
#print(model)
results = model.train(data=data_yaml_path, epochs=5, batch=32)

New https://pypi.org/project/ultralytics/8.3.175 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.174  Python-3.13.5 torch-2.7.1+cpu CPU (Intel Core(TM) i5-8365U 1.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/path/to/c2a_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train7, nbs=64, nms=False, opset=None, optim

train: Scanning D:\path\to\c2a_dataset\C2A_Dataset\new_dataset3\train\labels.cache... 6129 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6129/6129 [00:00<?, ?it/s]

train: D:\path\to\c2a_dataset\C2A_Dataset\new_dataset3\train\images\flood_image0407_3.png: 1 duplicate labels removed



c:\Users\Admin\miniconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


val: Fast image access  (ping: 0.10.0 ms, read: 193.3108.5 MB/s, size: 321.9 KB)


...

                   all       2043      72123      0.742      0.614      0.655       0.37

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5         0G      1.377     0.8799     0.9906        739        640: 100%|██████████| 192/192 [51:21<00:00, 16.05s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [06:55<00:00, 13.00s/it]


                   all       2043      72123      0.767      0.635      0.685        0.4

5 epochs completed in 4.991 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 6.2MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics 8.3.174  Python-3.13.5 torch-2.7.1+cpu CPU (Intel Core(TM) i5-8365U 1.60GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [04:48<00:00,  9.00s/it]


                   all       2043      72123      0.767      0.634      0.685        0.4
Speed: 2.3ms preprocess, 111.3ms inference, 0.0ms loss, 9.1ms postprocess per image
Results saved to runs\detect\train7


#Model Evaluation : Human detection

In [11]:
import matplotlib.pyplot as plt
import numpy as np

print("Evaluating Human Detection Model...")

yolo_results = model.val(data=data_yaml_path) 

# Print YOLO evaluation metrics
print("\n Human Detection Model Evaluation Complete!")
print(" YOLO Evaluation Metrics:")
mAP50 = yolo_results.results_dict.get('metrics/mAP50(B)', 'N/A')
mAP50_95 = yolo_results.results_dict.get('metrics/mAP50-95(B)', 'N/A')


print(f"  mAP@0.5: {mAP50:.4f}")
print(f"  mAP@0.5:0.95: {mAP50_95:.4f}")

# Plotting the evaluation metrics
metrics = ['mAP@0.5', 'mAP@0.5:0.95']
values = [mAP50, mAP50_95]

plt.figure(figsize=(8, 5))
plt.bar(metrics, values, color=['skyblue', 'lightgreen'])
plt.ylabel('Metric Value')
plt.title('YOLO Human Detection Model Evaluation Metrics')
plt.ylim(0, 1) # Metrics are typically in the range [0, 1]
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Evaluating Human Detection Model...
Ultralytics 8.3.174  Python-3.13.5 torch-2.7.1+cpu CPU (Intel Core(TM) i5-8365U 1.60GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1145.8302.6 MB/s, size: 600.1 KB)


val: Scanning D:\path\to\c2a_dataset\C2A_Dataset\new_dataset3\val\labels.cache... 2043 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2043/2043 [00:00<?, ?it/s]
c:\Users\Admin\miniconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [04:28<00:00,  4.19s/it]


                   all       2043      72123      0.767      0.634      0.685        0.4
Speed: 1.6ms preprocess, 99.1ms inference, 0.0ms loss, 6.7ms postprocess per image
Results saved to runs\detect\train72

 Human Detection Model Evaluation Complete!
 YOLO Evaluation Metrics:
  mAP@0.5: 0.6854
  mAP@0.5:0.95: 0.3999


<Figure size 800x500 with 1 Axes>

## Data preparation and preprocessing for disaster type classification

### Subtask:
Infer disaster types from image filenames or directory structure, define the classes for image classification based on inferred types, organize images into class-specific subfolders for training and validation, and prepare data loaders for the image classification model, including necessary preprocessing steps like resizing and normalization.


In [6]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define base paths
base_dataset_path = r'D:\c2a_dataset\C2A_Dataset\new_dataset3'
train_base_path = os.path.join(base_dataset_path, 'train')
val_base_path = os.path.join(base_dataset_path, 'val')

# Define disaster types (used for fixed class order in generators)
disaster_types = ['collapsed_building', 'flood', 'fire', 'traffic_incident']
print(f"Using disaster types for classification: {disaster_types}")

# -- IMAGE DATA GENERATORS (DATA LOADERS) --

# Define image preprocessing and augmentation for train/validation
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# These generators will index folders exactly as disaster_types are listed
train_generator = train_datagen.flow_from_directory(
    train_base_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=disaster_types
)

val_generator = val_datagen.flow_from_directory(
    val_base_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=disaster_types
)

print("\nTraining and validation data generators created.")
print(f"Training classes: {train_generator.class_indices}")
print(f"Validation classes: {val_generator.class_indices}")

# Function to preview a few samples from the data generator
def preview_samples(data_generator, n_batches=1):
    for i, (images, labels) in enumerate(data_generator):
        print(f"Batch {i+1}: images shape {images.shape}, labels shape {labels.shape}")
        if i+1 >= n_batches:
            break

# Preview a single batch of training data 
preview_samples(train_generator, n_batches=1)


c:\Users\Admin\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Admin\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Admin\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations i

Using disaster types for classification: ['collapsed_building', 'flood', 'fire', 'traffic_incident']
Found 6129 images belonging to 4 classes.
Found 2043 images belonging to 4 classes.

Training and validation data generators created.
Training classes: {'collapsed_building': 0, 'flood': 1, 'fire': 2, 'traffic_incident': 3}
Validation classes: {'collapsed_building': 0, 'flood': 1, 'fire': 2, 'traffic_incident': 3}
Batch 1: images shape (32, 224, 224, 3), labels shape (32, 4)


## Model building and Training: disaster type classification

### Subtask:
Choose a suitable image classification model architecture (e.g., a pre-trained model like ResNet or EfficientNet for transfer learning), load the pre-trained weights, and modify the final layers for the specific number of disaster classes.


In [12]:

# Create the model before compiling and training
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2


# Define the base pre-trained model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new Keras Functional API model on top of the frozen base model
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a final dense layer with num_classes units and softmax activation
num_classes = len(disaster_types)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model_classification = Model(inputs=base_model.input, outputs=predictions)


# Compile the model
model_classification.compile(optimizer='adam',
                             loss='categorical_crossentropy',
                             metrics=['accuracy', 'Precision', 'Recall', ])

# Train the model
num_epochs = 5
history = model_classification.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator
)

print("\nImage classification model training complete.")

Epoch 1/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - Precision: 0.8171 - Recall: 0.6722 - accuracy: 0.7414 - loss: 0.6273

c:\Users\Admin\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


192/192 ━━━━━━━━━━━━━━━━━━━━ 509s 3s/step - Precision: 0.9186 - Recall: 0.8319 - accuracy: 0.8706 - loss: 0.3522 - val_Precision: 0.9631 - val_Recall: 0.9334 - val_accuracy: 0.9496 - val_loss: 0.1625
Epoch 2/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 439s 2s/step - Precision: 0.9730 - Recall: 0.9522 - accuracy: 0.9635 - loss: 0.1284 - val_Precision: 0.9767 - val_Recall: 0.9623 - val_accuracy: 0.9697 - val_loss: 0.1056
Epoch 3/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 478s 2s/step - Precision: 0.9847 - Recall: 0.9745 - accuracy: 0.9791 - loss: 0.0830 - val_Precision: 0.9827 - val_Recall: 0.9750 - val_accuracy: 0.9785 - val_loss: 0.0812
Epoch 4/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 449s 2s/step - Precision: 0.9903 - Recall: 0.9830 - accuracy: 0.9873 - loss: 0.0586 - val_Precision: 0.9848 - val_Recall: 0.9804 - val_accuracy: 0.9834 - val_loss: 0.0672
Epoch 5/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 444s 2s/step - Precision: 0.9962 - Recall: 0.9928 - accuracy: 0.9949 - loss: 0.0441 - val_Precision: 0.9863 - val_Recall: 0.9843 - 

## Model Evaluation:disaster type classification



In [ ]:
import matplotlib.pyplot as plt

loss, accuracy = model_classification.evaluate(val_generator)

# Print classification evaluation metrics
print("\n Disaster Type Classification Model Evaluation Complete!")
print(" Classification Evaluation Metrics:")
# Plotting the evaluation metrics
metrics = ['Validation Loss', 'Validation Accuracy']
values = [loss, accuracy]

plt.figure(figsize=(8, 5))
plt.bar(metrics, values, color=['salmon', 'cornflowerblue'])
plt.ylabel('Metric Value')
plt.title('Disaster Type Classification Model Evaluation Metrics')
plt.ylim(0, max(loss, accuracy) + 0.1)
plt.show()

dict_keys(['Precision', 'Recall', 'accuracy', 'loss', 'val_Precision', 'val_Recall', 'val_accuracy', 'val_loss'])
64/64 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - Precision: 0.9862 - Recall: 0.9824 - accuracy: 0.9848 - loss: 0.0550

 Disaster Type Classification Model Evaluation Complete!
 Classification Evaluation Metrics:
  Validation Loss: 0.0550
  Validation Accuracy: 0.9848
  Validation Precision: 0.986240804195404
  Validation Recall: 0.9823788404464722


## Model Integration and Resource Recommendation Logic

### Subtask:
Integrate the two trained models (human detection and disaster type classification). Develop logic to analyze an image using both models and recommend appropriate emergency resources based on the detected human count and the classified disaster type.

In [14]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess_input 
import base64 
RESCUE_RESOURCES = {
    'collapsed_building': {
        'no_humans': ['Search and rescue dogs', 'Structural assessment team', 'Heavy equipment'],
        'low_density': ['Ground rescue teams (small)', 'Medical aid (basic)', 'Search and rescue dogs'],
        'medium_density': ['Ground rescue teams (multiple)', 'Medical aid (advanced)', 'Temporary shelter assessment'],
        'high_density': ['Multiple rescue teams (heavy urban search and rescue)', 'Medical triage setup', 'Temporary shelters', 'Food and water supplies']
    },
    'flood': {
        'no_humans': ['Flood damage assessment', 'Infrastructure check'],
        'low_density': ['Water rescue team (small boat)', 'Medical aid (basic)', 'Evacuation route assessment'],
        'medium_density': ['Water rescue teams (multiple boats)', 'Medical aid (advanced)', 'Evacuation support', 'Temporary shelters'],
        'high_density': ['Multiple water rescue teams (boats/helicopters)', 'Medical triage setup', 'Temporary shelters', 'Food and water supplies', 'Evacuation plans']
    },
    'fire': {
        'no_humans': ['Fire damage assessment', 'Safety perimeter setup'],
        'low_density': ['Firefighting units (local)', 'Medical aid (basic)'],
        'medium_density': ['Firefighting units (multiple)', 'Medical aid (advanced)', 'Evacuation route assessment'],
        'high_density': ['Multiple firefighting units (regional support)', 'Medical triage setup', 'Temporary shelters', 'Evacuation plans']
    },
    'traffic_incident': {
        'no_humans': ['Traffic management', 'Scene cleanup'],
        'low_density': ['Police/Emergency medical (basic)', 'Tow trucks'],
        'medium_density': ['Multiple police/emergency medical', 'Traffic management', 'Accident investigation'],
        'high_density': ['Major incident response team', 'Medical triage setup', 'Heavy recovery equipment']
    },
     'other': { # Default for uncategorized or unknown disaster types
        'no_humans': ['General assessment team'],
        'low_density': ['Basic rescue team', 'Medical aid (basic)'],
        'medium_density': ['Multiple rescue teams', 'Medical aid (advanced)'],
        'high_density': ['Major rescue operations', 'Medical triage setup']
    }
}

class IntegratedDisasterSystem:
    def __init__(self, yolo_model, classification_model, classification_class_names):
        self.yolo_model = yolo_model
        self.classification_model = classification_model
        self.classification_class_names = classification_class_names
        print("Integrated Disaster System Initialized")

    def classify_disaster_type(self, image_path):
        """Classify the type of disaster in the image"""
        img = keras_image.load_img(image_path, target_size=(self.classification_model.input_shape[1], self.classification_model.input_shape[2]))
        img_array = keras_image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        # Preprocess the image for ResNet
        img_array = resnet_preprocess_input(img_array)

        predictions = self.classification_model.predict(img_array, verbose=0)
        predicted_class_index = np.argmax(predictions[0])
        predicted_class_name = self.classification_class_names[predicted_class_index]
        confidence = np.max(predictions[0])

        return predicted_class_name, float(confidence)


    def detect_humans(self, image_path):
        """Detect humans in disaster scene using YOLO model"""

        results = self.yolo_model(image_path, conf=0.25, verbose=False)

        detections = []
        human_count = 0
        for r in results:
            boxes = r.boxes
            if boxes is not None:
                for box in boxes:
                        # 'human' is class 0 in the YOLO model
                    if box.cls[0] == 0:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        conf = box.conf[0].cpu().numpy()
                        detections.append({
                                'bbox': [int(x1), int(y1), int(x2), int(y2)],
                                'confidence': float(conf),
                                'center': [int((x1+x2)/2), int((y1+y2)/2)]
                            })
                        human_count += 1

        return detections, human_count


    def assess_human_density(self, human_count):
        """Assess human density category"""
        if human_count == 0:
            return 'no_humans'
        elif human_count <= 2:
            return 'low_density'
        elif human_count <= 5:
            return 'medium_density'
        else:
            return 'high_density'

    def recommend_resources(self, disaster_type, human_density_category):
        """Recommend resources based on disaster type and human density"""
        disaster_key = disaster_type.lower() if disaster_type.lower() in RESCUE_RESOURCES else 'other'
        density_key = human_density_category.lower() if human_density_category.lower() in RESCUE_RESOURCES[disaster_key] else 'no_humans'

        return RESCUE_RESOURCES[disaster_key][density_key]


    def analyze_scene(self, image_path):
        """Analyze a single disaster scene and provide recommendations"""
        print(f"\n--- Analyzing Scene: {os.path.basename(image_path)} ---")

        # 1. Classify Disaster Type
        disaster_type, type_confidence = self.classify_disaster_type(image_path)
        print(f" Classified Disaster Type: {disaster_type.upper()} (Confidence: {type_confidence:.2f})")

        # 2. Detect Humans
        detections, human_count = self.detect_humans(image_path)
        print(f"Humans Detected: {human_count}")
        print(f"Human Detections Details: {detections}") 

        # 3. Assess Human Density
        human_density_category = self.assess_human_density(human_count)
        print(f" Human Density Category: {human_density_category.replace('_', ' ').title()}")

        # 4. Recommend Resources
        recommended_resources = self.recommend_resources(disaster_type, human_density_category)
        print(f"Recommended Resources:")
        for i, resource in enumerate(recommended_resources, 1):
            print(f"   {i}. {resource}")
        
        # 5. image in the report (as base64)
        image_base64 = None
        try:
            with open(image_path, "rb") as img_file:
                image_base64 = base64.b64encode(img_file.read()).decode('utf-8')
        except Exception as e:
            print(f"Error encoding image to base64: {e}")

        # Return a structured report
        report = {
            'image_path': image_path,  
            'disaster_classification': {'type': disaster_type, 'confidence': type_confidence},
            'human_detection': {'count': human_count, 'detections': detections},
            'human_density_category': human_density_category,
            'recommended_resources': recommended_resources,
            'image_base64': image_base64  
        }
        return report



## Analysis and Visualization Functions

### Subtask:
Create functions to analyze a given image using the integrated system, generate a report, and visualize the results (e.g., bounding boxes for humans, overlaid disaster type).

In [16]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt


def visualize_disaster_analysis(image_path, analysis_report, save_path=None):
    
    img = cv2.imread(image_path)
    if img is None:
        print(f" Error loading image for visualization: {image_path}")
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, _ = img_rgb.shape

    # Draw bounding boxes for detected humans
    detections = analysis_report.get('human_detection', {}).get('detections', [])
    for det in detections:
        bbox = det['bbox']
        confidence = det['confidence']
        x1, y1, x2, y2 = bbox
        # Ensure coordinates are within image bounds
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width, x2), min(height, y2)
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2) # Green bounding box
        label = f"Human: {confidence:.2f}"
        cv2.putText(img_rgb, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

 
# ------- Random Image Selection and Visualization -------

# Path to the test images folder
test_images_folder = r'D:/c2a_dataset/C2A_Dataset/new_dataset3/test/images'

# List image files (case-insensitive match on common image formats)
image_files = [f for f in os.listdir(test_images_folder)
               if f.lower().endswith('.png')]
print(f"Found {len(image_files)} images in the test folder.")
print(f"Image files: {image_files[:5]}...")  # Display first 5 files for brevity

if not image_files:
    print("No images found in the specified test images folder!")
else:
    # Pick a random image
    random_image_file = random.choice(image_files)
    random_image_path = os.path.join(test_images_folder, random_image_file)
    random_image_path = random_image_path.replace("\\", "/")
    image_path = random_image_path
    print(f"Selected random image: {image_path}")

# Function to visualize the disaster analysis report
    integrated_system = IntegratedDisasterSystem(yolo_model = model, classification_model = model_classification, classification_class_names = disaster_types)

    analysis_report = integrated_system.analyze_scene(image_path) 
    visualize_disaster_analysis(random_image_path, analysis_report)


Found 2043 images in the test folder.
Image files: ['collapsed_building_image0001_3.png', 'collapsed_building_image0002_1.png', 'collapsed_building_image0002_3.png', 'collapsed_building_image0004_1.png', 'collapsed_building_image0005_2.png']...
Selected random image: D:/c2a_dataset/C2A_Dataset/new_dataset3/test/images/collapsed_building_image0139_1.png
Integrated Disaster System Initialized

--- Analyzing Scene: collapsed_building_image0139_1.png ---
 Classified Disaster Type: COLLAPSED_BUILDING (Confidence: 0.97)
Humans Detected: 21
Human Detections Details: [{'bbox': [349, 66, 376, 86], 'confidence': 0.8836973905563354, 'center': [362, 76]}, {'bbox': [320, 252, 351, 268], 'confidence': 0.8163034915924072, 'center': [335, 260]}, {'bbox': [331, 30, 340, 53], 'confidence': 0.8050060868263245, 'center': [336, 41]}, {'bbox': [59, 53, 74, 84], 'confidence': 0.8020620346069336, 'center': [66, 69]}, {'bbox': [248, 106, 266, 119], 'confidence': 0.7907145023345947, 'center': [257, 112]}, {'bbo

## System Demo

### Subtask:
Develop a demo to showcase the complete system's capabilities using sample disaster images.

In [21]:
import os
import json
import matplotlib.pyplot as plt 
import cv2 
import random 

def process_disaster_area(folder_path, max_images=None):

    print(f" PROCESSING DISASTER AREA: {folder_path}")
    print("=" * 60)

    image_files = []
    if os.path.exists(folder_path):
        for file in os.listdir(folder_path):
            # Check for common image file extensions
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                image_files.append(os.path.join(folder_path, file))
        # image_files.sort() # Removed sorting for random selection

    else:
        print(f" Error: Folder not found at {folder_path}")
        return []

    if not image_files:
        print(f"⚠️ No image files found in {folder_path}")
        return []

    # Shuffle the image files for random selection
    random.shuffle(image_files)

    # Limit the number of images if max_images is specified
    if max_images is not None:
        # Ensure max_images does not exceed the total number of images
        max_images = min(max_images, len(image_files))
        image_files = image_files[:max_images]
        print(f"Processing {len(image_files)} randomly selected images as requested.")


    # Process each image
    analysis_reports = []
    print(f"Found {len(image_files)} images to process.")
    for i, img_path in enumerate(image_files):
        print(f"\n--- Processing Image {i+1}/{len(image_files)}: {os.path.basename(img_path)} ---")
        try:
            report = integrated_system.analyze_scene(img_path)
            if report:
                analysis_reports.append(report)
                # Display the image after analysis
                img = cv2.imread(img_path)
                if img is not None:
                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    plt.figure(figsize=(6, 4)) 
                    plt.imshow(img_rgb)
                    plt.title(f"Processed: {os.path.basename(img_path)}")
                    plt.axis('off')
                    plt.show()
                else:
                    print(f" Could not load image for display: {os.path.basename(img_path)}")

        except Exception as e:
            print(f" Error processing image {os.path.basename(img_path)}: {e}")





    return analysis_reports

sample_folder_path = r'D:/c2a_dataset/C2A_Dataset/new_dataset3/test/images'
area_analysis_results = process_disaster_area(sample_folder_path, max_images=5) 

 PROCESSING DISASTER AREA: D:/c2a_dataset/C2A_Dataset/new_dataset3/test/images
Processing 5 randomly selected images as requested.
Found 5 images to process.

--- Processing Image 1/5: flood_image0319_3.png ---

--- Analyzing Scene: flood_image0319_3.png ---
 Classified Disaster Type: FLOOD (Confidence: 1.00)
Humans Detected: 27
Human Detections Details: [{'bbox': [403, 350, 429, 375], 'confidence': 0.8069194555282593, 'center': [416, 363]}, {'bbox': [606, 173, 630, 191], 'confidence': 0.7849034667015076, 'center': [618, 182]}, {'bbox': [314, 572, 339, 591], 'confidence': 0.7820339202880859, 'center': [326, 582]}, {'bbox': [300, 255, 324, 275], 'confidence': 0.7300440669059753, 'center': [312, 265]}, {'bbox': [302, 702, 324, 727], 'confidence': 0.7281609773635864, 'center': [313, 714]}, {'bbox': [653, 578, 668, 603], 'confidence': 0.6974872946739197, 'center': [660, 591]}, {'bbox': [911, 98, 920, 116], 'confidence': 0.616899311542511, 'center': [916, 107]}, {'bbox': [597, 58, 623, 79],

<Figure size 600x400 with 1 Axes>


--- Processing Image 2/5: traffic_incident_image0446_2.png ---

--- Analyzing Scene: traffic_incident_image0446_2.png ---
 Classified Disaster Type: TRAFFIC_INCIDENT (Confidence: 0.86)
Humans Detected: 44
Human Detections Details: [{'bbox': [694, 423, 720, 444], 'confidence': 0.8897274136543274, 'center': [707, 434]}, {'bbox': [543, 61, 569, 80], 'confidence': 0.8840972185134888, 'center': [556, 70]}, {'bbox': [5, 623, 36, 640], 'confidence': 0.8533737659454346, 'center': [21, 631]}, {'bbox': [141, 595, 172, 614], 'confidence': 0.8300378918647766, 'center': [156, 604]}, {'bbox': [59, 633, 77, 673], 'confidence': 0.7921713590621948, 'center': [68, 653]}, {'bbox': [610, 668, 636, 682], 'confidence': 0.788067102432251, 'center': [623, 675]}, {'bbox': [370, 219, 389, 239], 'confidence': 0.7823704481124878, 'center': [379, 229]}, {'bbox': [168, 56, 198, 72], 'confidence': 0.7673643827438354, 'center': [183, 64]}, {'bbox': [241, 658, 265, 682], 'confidence': 0.7376306056976318, 'center': [2

<Figure size 600x400 with 1 Axes>


--- Processing Image 3/5: collapsed_building_image0031_3.png ---

--- Analyzing Scene: collapsed_building_image0031_3.png ---
 Classified Disaster Type: COLLAPSED_BUILDING (Confidence: 0.95)
Humans Detected: 38
Human Detections Details: [{'bbox': [132, 182, 155, 205], 'confidence': 0.926459789276123, 'center': [144, 194]}, {'bbox': [245, 165, 264, 195], 'confidence': 0.8956925868988037, 'center': [255, 180]}, {'bbox': [95, 49, 127, 66], 'confidence': 0.8953154683113098, 'center': [111, 58]}, {'bbox': [171, 245, 185, 277], 'confidence': 0.8900136351585388, 'center': [178, 261]}, {'bbox': [246, 266, 269, 291], 'confidence': 0.8705659508705139, 'center': [257, 278]}, {'bbox': [40, 278, 55, 291], 'confidence': 0.8316388726234436, 'center': [47, 285]}, {'bbox': [81, 12, 97, 42], 'confidence': 0.8266171216964722, 'center': [89, 27]}, {'bbox': [144, 236, 164, 249], 'confidence': 0.826376736164093, 'center': [154, 243]}, {'bbox': [40, 47, 67, 68], 'confidence': 0.8254494071006775, 'center': [

<Figure size 600x400 with 1 Axes>


--- Processing Image 4/5: fire_image0416_1.png ---

--- Analyzing Scene: fire_image0416_1.png ---
 Classified Disaster Type: FIRE (Confidence: 1.00)
Humans Detected: 36
Human Detections Details: [{'bbox': [31, 184, 53, 204], 'confidence': 0.9397034049034119, 'center': [42, 194]}, {'bbox': [5, 160, 30, 178], 'confidence': 0.924670934677124, 'center': [18, 169]}, {'bbox': [188, 104, 200, 130], 'confidence': 0.8979839086532593, 'center': [194, 117]}, {'bbox': [6, 44, 29, 65], 'confidence': 0.8849570751190186, 'center': [18, 54]}, {'bbox': [223, 228, 238, 246], 'confidence': 0.8822351098060608, 'center': [231, 237]}, {'bbox': [216, 120, 233, 132], 'confidence': 0.8091199994087219, 'center': [224, 126]}, {'bbox': [0, 95, 16, 112], 'confidence': 0.78221595287323, 'center': [8, 104]}, {'bbox': [183, 73, 198, 88], 'confidence': 0.7771599292755127, 'center': [191, 80]}, {'bbox': [6, 114, 23, 138], 'confidence': 0.7591658234596252, 'center': [15, 126]}, {'bbox': [16, 21, 31, 31], 'confidence': 

<Figure size 600x400 with 1 Axes>


--- Processing Image 5/5: fire_image0182_3.png ---

--- Analyzing Scene: fire_image0182_3.png ---
 Classified Disaster Type: FIRE (Confidence: 1.00)
Humans Detected: 30
Human Detections Details: [{'bbox': [277, 347, 307, 365], 'confidence': 0.8965851068496704, 'center': [292, 356]}, {'bbox': [39, 362, 61, 385], 'confidence': 0.8939927816390991, 'center': [50, 374]}, {'bbox': [448, 339, 460, 363], 'confidence': 0.8641926050186157, 'center': [454, 351]}, {'bbox': [345, 73, 364, 98], 'confidence': 0.8518361449241638, 'center': [355, 86]}, {'bbox': [247, 328, 267, 348], 'confidence': 0.8465059995651245, 'center': [257, 338]}, {'bbox': [352, 34, 385, 51], 'confidence': 0.8145831823348999, 'center': [369, 43]}, {'bbox': [70, 252, 89, 283], 'confidence': 0.8052578568458557, 'center': [80, 268]}, {'bbox': [170, 141, 186, 156], 'confidence': 0.7983502149581909, 'center': [178, 148]}, {'bbox': [300, 220, 318, 243], 'confidence': 0.7940569519996643, 'center': [309, 231]}, {'bbox': [337, 10, 361,

<Figure size 600x400 with 1 Axes>

# Task
Deploy the trained YOLO human detection model and the Keras disaster type classification model.

## Refine and save models

### Subtask:
Save the trained YOLO human detection model and the Keras disaster type classification model in a deployable format (e.g., TensorFlow SavedModel, ONNX, or a format suitable for a specific deployment platform).


In [22]:
import os

# Define paths to save the models
yolo_save_path = r'D:/model_yolo.pt' # YOLOv8 native format
classification_save_path = r'D:/resnet50v2_disaster_classification_savedmodel.h5' # TensorFlow SavedModel format

# 1. Save the trained YOLO model
try:
    model.save(yolo_save_path)
    print(f"YOLO human detection model saved to: {yolo_save_path}")
except Exception as e:
    print(f"Error saving YOLO model: {e}")


# 2. Save the trained Keras classification model in TensorFlow SavedModel format
try:
    model_classification.save(classification_save_path)
    print(f"Keras disaster classification model saved to: {classification_save_path}")
except Exception as e:
    print(f"Error saving Keras classification model: {e}")


YOLO human detection model saved to: D:/model_yolo.pt
Keras disaster classification model saved to: D:/resnet50v2_disaster_classification_savedmodel.h5


## Deploy and test (Flask Application)

### Subtask:
Deploy the Flask application (e.g., using ngrok in Colab for testing, or to a cloud platform) and test it with new disaster images to ensure it functions correctly and provides timely recommendations.

In [ ]:
import os
import tempfile
from flask import Flask, request, jsonify, render_template_string
import tensorflow as tf
from ultralytics import YOLO
import numpy as np
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess_input
import base64

# Rescue resource recommendations
RESCUE_RESOURCES = {
    'collapsed_building': {
        'no_humans': ['Search and rescue dogs', 'Structural assessment team', 'Heavy equipment'],
        'low_density': ['Ground rescue teams (small)', 'Medical aid (basic)', 'Search and rescue dogs'],
        'medium_density': ['Ground rescue teams (multiple)', 'Medical aid (advanced)', 'Temporary shelter assessment'],
        'high_density': ['Multiple rescue teams (heavy urban search and rescue)', 'Medical triage setup', 'Temporary shelters', 'Food and water supplies']
    },
    'flood': {
        'no_humans': ['Flood damage assessment', 'Infrastructure check'],
        'low_density': ['Water rescue team (small boat)', 'Medical aid (basic)', 'Evacuation route assessment'],
        'medium_density': ['Water rescue teams (multiple boats)', 'Medical aid (advanced)', 'Evacuation support', 'Temporary shelters'],
        'high_density': ['Multiple water rescue teams (boats/helicopters)', 'Medical triage setup', 'Temporary shelters', 'Food and water supplies', 'Evacuation plans']
    },
    'fire': {
        'no_humans': ['Fire damage assessment', 'Safety perimeter setup'],
        'low_density': ['Firefighting units (local)', 'Medical aid (basic)'],
        'medium_density': ['Firefighting units (multiple)', 'Medical aid (advanced)', 'Evacuation route assessment'],
        'high_density': ['Multiple firefighting units (regional support)', 'Medical triage setup', 'Temporary shelters', 'Evacuation plans']
    },
    'traffic_incident': {
        'no_humans': ['Traffic management', 'Scene cleanup'],
        'low_density': ['Police/Emergency medical (basic)', 'Tow trucks'],
        'medium_density': ['Multiple police/emergency medical', 'Traffic management', 'Accident investigation'],
        'high_density': ['Major incident response team', 'Medical triage setup', 'Heavy recovery equipment']
    },
    'other': {
        'no_humans': ['General assessment team'],
        'low_density': ['Basic rescue team', 'Medical aid (basic)'],
        'medium_density': ['Multiple rescue teams', 'Medical aid (advanced)'],
        'high_density': ['Major rescue operations', 'Medical triage setup']
    }
}

# Integrated Disaster System Class
class IntegratedDisasterSystem:
    def __init__(self, yolo_model, classification_model, classification_class_names):
        self.yolo_model = yolo_model
        self.classification_model = classification_model
        self.classification_class_names = classification_class_names
        print("Integrated Disaster System Initialized")

    def classify_disaster_type(self, image_path):
        """Classify the type of disaster in the image"""
        img = keras_image.load_img(image_path, target_size=(self.classification_model.input_shape[1], self.classification_model.input_shape[2]))
        img_array = keras_image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        # Assume resnet-style preprocessing
        img_array = resnet_preprocess_input(img_array)
        predictions = self.classification_model.predict(img_array, verbose=0)
        predicted_class_index = np.argmax(predictions[0])
        predicted_class_name = self.classification_class_names[predicted_class_index]
        confidence = np.max(predictions[0])
        return predicted_class_name, float(confidence)

    def detect_humans(self, image_path):
        """Detect humans in disaster scene using YOLO model"""
        results = self.yolo_model(image_path, conf=0.25, verbose=False)
        detections = []
        human_count = 0
        for r in results:
            boxes = r.boxes
            if boxes is not None:
                for box in boxes:
                    # 'human' assumed class 0 in YOLO
                    if box.cls[0] == 0:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        conf = box.conf[0].cpu().numpy()
                        detections.append({
                            'bbox': [int(x1), int(y1), int(x2), int(y2)],
                            'confidence': float(conf),
                            'center': [int((x1+x2)/2), int((y1+y2)/2)]
                        })
                        human_count += 1
        return detections, human_count

    def assess_human_density(self, human_count):
        """Assess human density category"""
        if human_count == 0:
            return 'no_humans'
        elif human_count <= 2:
            return 'low_density'
        elif human_count <= 5:
            return 'medium_density'
        else:
            return 'high_density'

    def recommend_resources(self, disaster_type, human_density_category):
        """Recommend resources based on disaster type and human density"""
        disaster_key = disaster_type.lower() if disaster_type.lower() in RESCUE_RESOURCES else 'other'
        density_key = human_density_category.lower() if human_density_category.lower() in RESCUE_RESOURCES[disaster_key] else 'no_humans'
        return RESCUE_RESOURCES[disaster_key][density_key]

    def analyze_scene(self, image_path):
        """Analyze a single disaster scene and provide recommendations"""
        print(f"\n--- Analyzing Scene: {os.path.basename(image_path)} ---")
        disaster_type, type_confidence = self.classify_disaster_type(image_path)
        print(f" Classified Disaster Type: {disaster_type.upper()} (Confidence: {type_confidence:.2f})")
        detections, human_count = self.detect_humans(image_path)
        print(f"Humans Detected: {human_count}")
        human_density_category = self.assess_human_density(human_count)
        print(f" Human Density Category: {human_density_category.replace('_', ' ').title()}")
        recommended_resources = self.recommend_resources(disaster_type, human_density_category)
        print("Recommended Resources:")
        for i, resource in enumerate(recommended_resources, 1):
            print(f"   {i}. {resource}")
        image_base64 = None
        try:
            with open(image_path, "rb") as img_file:
                image_base64 = base64.b64encode(img_file.read()).decode('utf-8')
        except Exception as e:
            print(f"Error encoding image to base64: {e}")
        report = {
            'image_path': image_path,
            'disaster_classification': {'type': disaster_type, 'confidence': type_confidence},
            'human_detection': {'count': human_count, 'detections': detections},
            'human_density_category': human_density_category,
            'recommended_resources': recommended_resources,
            'image_base64': image_base64
        }
        return report

# Model Loading Paths
YOLO_MODEL_PATH = r"D:/model_yolo.pt"
CLASSIFICATION_MODEL_PATH = r"D:/resnet50v2_disaster_classification_savedmodel.h5"

# Load YOLO model
try:
    yolo_model = YOLO(YOLO_MODEL_PATH)
except Exception as e:
    print(f"Failed to load YOLO model: {e}")
    yolo_model = None

# Load classification model
try:
    classification_model = tf.keras.models.load_model(CLASSIFICATION_MODEL_PATH)
except Exception as e:
    print(f"Failed to load classification model: {e}")
    classification_model = None

disaster_types = ['collapsed_building', 'flood', 'fire', 'traffic_incident']

if yolo_model and classification_model:
    integrated_system = IntegratedDisasterSystem(yolo_model, classification_model, disaster_types)
else:
    integrated_system = None

# Flask App Setup
app = Flask(__name__)

UPLOAD_HTML = """
<!doctype html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Disaster Image Analysis</title>
  <style>
    body {
      background: linear-gradient(135deg, #89f7fe 0%, #66a6ff 100%);
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
      color: #222;
      margin: 40px auto;
      max-width: 700px;
      padding: 20px;
      border-radius: 15px;
      box-shadow: 0 8px 16px rgba(0,0,0,0.3);
      background-color: white;
    }
    h1 {
      color: #0056b3;
      text-align: center;
      margin-bottom: 30px;
    }
    form {
      display: flex;
      flex-direction: column;
      align-items: center;
      margin-bottom: 30px;
    }
    input[type="file"] {
      padding: 10px;
      border: 2px dashed #0056b3;
      border-radius: 10px;
      width: 90%;
      max-width: 350px;
      margin-bottom: 15px;
      cursor: pointer;
      transition: border-color 0.3s ease;
    }
    input[type="file"]:hover {
      border-color: #003d75;
    }
    input[type="submit"] {
      background-color: #0056b3;
      color: white;
      font-weight: bold;
      border: none;
      border-radius: 10px;
      padding: 12px 25px;
      cursor: pointer;
      transition: background-color 0.3s ease;
    }
    input[type="submit"]:hover {
      background-color: #003d75;
    }
    .result {
      background-color: #f0f7ff;
      border: 2px solid #66a6ff;
      border-radius: 10px;
      padding: 20px;
      white-space: pre-wrap;
      font-family: 'Courier New', Courier, monospace;
      font-size: 1rem;
      color: #003d75;
    }
    .result h2 {
      color: #0056b3;
      text-align: center;
      margin-bottom: 20px;
    }
    .line {
      margin-bottom: 10px;
      padding-left: 10px;
      border-left: 3px solid #0056b3;
    }
    img.uploaded-image {
      max-width: 100%;
      height: auto;
      border-radius: 10px;
      display: block;
      margin: 20px auto;
      box-shadow: 0 4px 12px rgba(0,0,0,0.15);
    }
  </style>
</head>
<body>
  <h1>Upload an image for disaster analysis</h1>
  <form method="post" enctype="multipart/form-data">
    <input type="file" name="image" accept="image/png, image/jpeg">
    <input type="submit" value="Analyze">
  </form>

  {% if result_lines %}
  <div class="result">
    <h2>Result</h2>
    {% for line in result_lines %}
      <div class="line">{{ line }}</div>
    {% endfor %}
  </div>
  {% endif %}

  {% if image_base64 %}
  <img class="uploaded-image" src="data:image/png;base64,{{ image_base64 }}" alt="Uploaded Image" />
  {% endif %}
</body>
</html>
"""

@app.route('/', methods=['GET'])
def home():
    return '<a href="/upload">Go to interactive disaster analysis page</a>'

@app.route('/upload', methods=['GET', 'POST'])
def upload_page():
    result_lines = None
    image_base64 = None
    temp_path = None
    if request.method == 'POST':
        if 'image' not in request.files or request.files['image'].filename == '':
            result_lines = ["No file selected!"]
        else:
            file = request.files['image']
            try:
                with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as temp:
                    temp_path = temp.name
                    file.save(temp_path)
                if integrated_system:
                    report = integrated_system.analyze_scene(temp_path)
                    result_lines = [
                        f"Disaster Type: {report['disaster_classification']['type']} (Confidence: {report['disaster_classification']['confidence']:.2f})",
                        f"Human Count: {report['human_detection']['count']}",
                        f"Human Density Category: {report['human_density_category'].replace('_', ' ').title()}",
                        "Resource Recommendations:"
                    ] + [f"  - {res}" for res in report['recommended_resources']]
                    image_base64 = report.get('image_base64', None)
                else:
                    result_lines = ["System not initialized."]
            except Exception as ex:
                result_lines = [f"Analysis failed. {ex}"]
            finally:
                if temp_path and os.path.exists(temp_path):
                    os.remove(temp_path)
    return render_template_string(UPLOAD_HTML, result_lines=result_lines, image_base64=image_base64)

@app.route('/analyze', methods=['POST'])
def analyze_image():
    temp_path = None
    if integrated_system is None:
        return jsonify({"error": "System not initialized properly."}), 500
    if 'image' not in request.files:
        return jsonify({"error": "No image file provided."}), 400
    file = request.files['image']
    if file.filename == '':
        return jsonify({"error": "No selected file."}), 400
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as temp:
            temp_path = temp.name
            file.save(temp_path)
        report = integrated_system.analyze_scene(temp_path)
        return jsonify(report), 200
    except Exception as ex:
        return jsonify({"error": "Analysis failed.", "details": str(ex)}), 500
    finally:
        if temp_path and os.path.exists(temp_path):
            os.remove(temp_path)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


Integrated Disaster System Initialized
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.113:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.0.113 - - [08/Aug/2025 10:16:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:192.168.0.113 - - [08/Aug/2025 10:16:50] "GET /upload HTTP/1.1" 200 -



--- Analyzing Scene: tmpmy9ci2q4.png ---


INFO:werkzeug:192.168.0.113 - - [08/Aug/2025 10:17:02] "POST /upload HTTP/1.1" 200 -


 Classified Disaster Type: COLLAPSED_BUILDING (Confidence: 1.00)
Humans Detected: 17
 Human Density Category: High Density
Recommended Resources:
   1. Multiple rescue teams (heavy urban search and rescue)
   2. Medical triage setup
   3. Temporary shelters
   4. Food and water supplies


INFO:werkzeug:192.168.0.113 - - [08/Aug/2025 10:17:16] "POST /upload HTTP/1.1" 200 -



--- Analyzing Scene: tmpgu9bkua5.png ---
 Classified Disaster Type: COLLAPSED_BUILDING (Confidence: 0.99)
Humans Detected: 40
 Human Density Category: High Density
Recommended Resources:
   1. Multiple rescue teams (heavy urban search and rescue)
   2. Medical triage setup
   3. Temporary shelters
   4. Food and water supplies


INFO:werkzeug:192.168.0.113 - - [08/Aug/2025 10:17:25] "POST /upload HTTP/1.1" 200 -
